In [ ]:
options(htmltools.dir.version = FALSE)
library(tidyverse)
# source('xaringan2pdf.R')
# xaringan_to_pdf('Chapter4.html')

## Data wrangling: `dplyr`

> Data scientists, according to interviews and expert estimates, spend from 50 percent to 80 percent of their time mired in the mundane labor of collecting and preparing data, before it can be explored for useful information. - NYTimes (2014)

What are some common things you like to do with your data? Maybe remove rows or columns, do calculations and maybe add new columns? This is called **data wrangling**. It’s not data management or data manipulation: you **keep the raw data raw** and do these things programatically in R with the tidyverse.

We are going to introduce you to data wrangling in R first with the tidyverse. The tidyverse is a suite of packages that match a philosophy of data science developed by Hadley Wickham and the RStudio team. I find it to be a more straight-forward way to learn R. We will also show you by comparison what code will look like in “Base R”.

---
### Objectives
- discuss tidy data
- read data from online into R
- explore gapminder data with base-R functions
- wrangle gapminder data with dplyr tidyverse functions
- practice RStudio-GitHub workflow

### Data and packages
**Gapminder data**

We’ll be using Gapminder data, which represents the health and wealth of nations. It was pioneered by Hans Rosling, who is famous for describing the prosperity of nations over time through famines, wars and other historic events with this beautiful data visualization in his [2006 TED Talk: The best stats you’ve ever seen](https://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen):
---
#### [Gapminder Motion Chart](http://www.gapminder.org/world)

In [ ]:
knitr::include_graphics('figures/gapminder_motion.png')

We’ll use the package `dplyr`, which is bundled within the `tidyverse` package. Please install the `tidyverse` ahead of time:

In [ ]:
install.packages("tidyverse")

---
### Tidy Data
Let’s start off discussing Tidy Data.

Hadley Wickham, RStudio’s Chief Scientist, and his team have been building R packages for data wrangling and visualization based on the idea of tidy data.

Tidy data has a simple convention: put variables in the columns and observations in the rows.

In [ ]:
knitr::include_graphics('figures/tidy_data.png')

---
When data are tidy, you are set up to work with it for your analyses, plots, etc.

In [ ]:
knitr::include_graphics('figures/tidy_img_np.png')

Right now we are going to use `dplyr` to wrangle this tidy-ish data set (the transform part of the cycle), and then come back to tidying messy data using `tidyr` once we’ve had some fun wrangling. These are both part of the `tidyverse` package that we’ve already installed.
---

In [ ]:
knitr::include_graphics('figures/r4ds_data-science.png')

Conceptually, making data tidy first is really critical. Instead of building your analyses around whatever (likely weird) format your data are in, take deliberate steps to make your data tidy. When your data are tidy, you can use a growing assortment of powerful analytical and visualization tools instead of inventing home-grown ways to accommodate your data. This will save you time since you aren’t reinventing the wheel, and will make your work more clear and understandable to your collaborators (most importantly, Future You).

---
#### Setup
We’ll do this in a new RMarkdown file.

Here’s what to do:

Clear your workspace (Session > Restart R)
1. New File > R Markdown
2. Save as `gapminder-wrangle.Rmd`
3. Delete the irrelevant text and write a little note to yourself about how we’ll be wrangling gapminder data using `dplyr`. You can edit the title too if you need to.
---
#### load `tidyverse` (which has `dplyr` inside)
In your R Markdown file, let’s make sure we’ve got our libraries loaded. Write the following:

In [ ]:
library(tidyverse)     ## install.packages("tidyverse")

This is becoming standard practice for how to load a library in a file, and if you get an error that the library doesn’t exist, you can install the package easily by running the code within the comment (highlight `install.packages("tidyverse")` and run it).

---
### Explore the gapminder data.frame
#### read data with `readr::read_csv()`

In our R Markdown, let’s read this csv file and name the variable “gapminder”. We will use the `read_csv()` function from the `readr` package (part of the tidyverse, so it’s already installed!).

In [ ]:
## read gapminder csv. Note the readr:: prefix identifies which package it's in
gapminder <- readr::read_csv('data/gapminder.csv') 

Let’s inspect:

In [ ]:
View(gapminder)

Let’s use head and tail:

In [ ]:
head(gapminder) # shows first 6

---

In [ ]:
tail(gapminder) # shows last 6
head(gapminder, 10) # shows first X that you indicate

---
`str()` will provide a sensible description of almost anything: when in doubt, inspect using str() on some of the recently created objects to get some ideas about what to do next.

In [ ]:
str(gapminder) # ?str - displays the structure of an object

---
`gapminder` is a `data.frame`. It is also a `tibble`, a modern extended structure based on data.frame. 

- Tibbles are data.frames but modify some older behaviours to make life a little easier

- Preferred data format in the tidyverse

- No need to worry about this!
---
### `dplyr` basics

There are five `dplyr` functions that you will use to do the vast majority of data manipulations:

- `filter()`: pick observations by their values
- `select()`: pick variables by their names
- `mutate()`: create new variables with functions of existing variables
- `summarise()`: collapse many values down to a single summary
- `arrange()`: reorder the rows

These can all be used in conjunction with `group_by()` which changes the scope of each function from operating on the entire dataset to operating on it group-by-group. These six functions provide the verbs for a language of data manipulation.

All verbs work similarly:

1. The first argument is a data frame.
2. The subsequent arguments describe what to do with the data frame. You can refer to columns in the data frame directly without using `$`.
3. The result is a new data frame.

Together these properties make it easy to chain together multiple simple steps to achieve a complex result.
---
#### `filter()` subsets data row-wise (observations)
You will want to isolate bits of your data; maybe you want to only look at a single country or a few years. R calls this subsetting.

`filter()` is a function in dplyr that takes logical expressions and returns the rows for which all are TRUE.

Visually, we are doing this (thanks RStudio for your cheatsheet):

In [ ]:
knitr::include_graphics('figures/rstudio-cheatsheet-filter.png')

---
Remember your logical expressions? Conditional operators accepted in filter: `==, <, >, <=, >=, is.na(), !is.na(), %in%, !, |, &, xor()`

In [ ]:
filter(gapminder, lifeExp < 29)

You can say this out loud: “Filter the gapminder data for life expectancy less than 29”. Notice that when we do this, all the columns are returned, but only the rows that have the life expectancy less than 29. We’ve subsetted by row.
---
Let’s try another: “Filter the gapminder data for the country Mexico”.

In [ ]:
filter(gapminder, country == "Mexico")

---
How about if we want two country names? We can’t use the `==` operator here, because it can only operate on one thing at a time. We will use the `%in%` operator:

In [ ]:
filter(gapminder, country %in% c("Mexico", "Peru"))

---
How about if we want Mexico in 2002? You can pass filter different criteria:

In [ ]:
filter(gapminder, country == "Mexico", year == 2002)

#### Exercise

What was the average life expectency in Brazil between 1987 and 2007?
Hint: do this in 2 steps by assigning a variable and then using the `mean()` function.

In [ ]:
x <- filter(gapminder, country == "Brazil", year > 1986)  
mean(x$lifeExp)  

Then, sync to Github.com (pull, stage, commit, push).
---
### `select()` subsets data column-wise (variables)
We use `select()` to subset the data on variables or columns.

Visually, we are doing this (thanks RStudio for your cheatsheet):

In [ ]:
knitr::include_graphics('figures/rstudio-cheatsheet-select.png')

---
- We can select multiple columns with a comma, after we specify the data frame (gapminder).

In [ ]:
gap1 <- dplyr::select(gapminder, year, country, lifeExp) 
head(gap1, 3)

- We can select a range of variables with a semicolon.

In [ ]:
gap2 <- dplyr::select(gapminder, year:lifeExp) 
head(gap2, 3)

---
- We can select columns with indices.

In [ ]:
gap3 <- dplyr::select(gapminder, 1, 2, 4) 
head(gap3, 3)

- We can also use `-` to deselect columns

In [ ]:
gap4 <- dplyr::select(gapminder, -continent, -lifeExp) # you can use - to deselect columns
head(gap4, 3)

---
### Use `select()` and `filter()` together
Let’s filter for Cambodia and remove the continent and lifeExp columns. We’ll save this as a variable. Actually, as two temporary variables, which means that for the second one we need to operate on `gap_cambodia`, not `gapminder`.

In [ ]:
gap_cambodia  <- filter(gapminder, country == "Cambodia")
gap_cambodia2 <- dplyr::select(gap_cambodia, -continent, -lifeExp) 

We also could have called them both `gap_cambodia` and overwritten the first assignment. Either way, naming them and keeping track of them gets super cumbersome, which means more time to understand what’s going on and opportunities for confusion or error.
---
### Meet the new pipe `%>%` (`|>`) operator

Before we go any further, we should explore the new pipe operator that `dplyr` imports from the `magrittr` package by Stefan Bache. This is going to **change your life**. You no longer need to enact multi-operation commands by nesting them inside each other. And we won’t need to make temporary variables like we did in the Cambodia example above. This new syntax leads to code that is much easier to write and to read: it actually tells the story of your analysis.

Here’s what it looks like: `%>%`. The RStudio keyboard shortcut: Ctrl + Shift + M (Windows), Cmd + Shift + M (Mac). Or you can use the base R pipe `|>` built into R 4.1 and later version. Go to the Global Options window, select "Code" and you'll see an option for "use native pipe operator, |>". 

Let’s demo then I’ll explain:

In [ ]:
gapminder |> head(3)

---
This is equivalent to `head(gapminder, 3)`. This pipe operator takes the thing on the left-hand-side and **pipes** it into the function call on the right-hand-side. It literally drops it in as the first argument.

**You should think “and then” whenever you see the pipe operator, `|>`.**

One of the most awesome things about this is that you START with the data before you say what you’re doing to DO to it. So above: “take the gapminder data, and then give me the first three entries”.

This means that instead of this:

In [ ]:
## instead of this...
gap_cambodia  <- filter(gapminder, country == "Cambodia")
gap_cambodia2 <- dplyr::select(gap_cambodia, -continent, -lifeExp) 

## ...we can do this
gap_cambodia  <- gapminder |> filter(country == "Cambodia")
gap_cambodia2 <- gap_cambodia |> dplyr::select(-continent, -lifeExp) 

So you can see that we’ll start with gapminder in the first example line, and then `gap_cambodia` in the second. This makes it a bit easier to see what data we are starting with and what we are doing to it.

But, we still have those temporary variables so we’re not truly that better off. But get ready to be majorly impressed:
---
### Revel in the convenience
We can use the pipe to chain those two operations together:

In [ ]:
gap_cambodia  <- gapminder |> 
  filter(country == "Cambodia") |>
  dplyr::select(-continent, -lifeExp) 

What’s happening here? In the second line, we were able to delete `gap_cambodia2 <- gap_cambodia`, and put the pipe operator above. This is possible since we wanted to operate on the `gap_cambodia` data anyways. And we weren’t truly excited about having a second variable named `gap_cambodia2` anyways, so we can get rid of it. This is huge, because most of your data wrangling will have many more than 2 steps, and we don’t want a `gap_cambodia14`!
---
By using multiple lines I can actually read this like a story and there aren’t temporary variables that get super confusing. In my head:

>“start with the gapminder data, and then filter for Cambodia, and then drop the variables continent and lifeExp.”

Being able to read a story out of code like this is really game-changing. We’ll continue using this syntax as we learn the other dplyr verbs.
---
### `mutate()` adds new variables
Let’s say we needed to add an index column so we know which order these data came in. Let’s not make a new variable, let’s add a column to our gapminder data frame. How do we do that? With the ·mutate()· function.

Visually, we are doing this (thanks RStudio for your cheatsheet):

In [ ]:
knitr::include_graphics('figures/rstudio-cheatsheet-mutate.png')

---
Imagine we want to know each country’s annual GDP. We can multiply pop by gdpPercap to create a new column named gdp.

In [ ]:
gapminder |>
  mutate(gdp = pop * gdpPercap)

#### Your turn

> Calculate the population in thousands for all Asian countries in the year 2007 and add it as a new column.

In [ ]:
gapminder |>
  filter(continent == "Asia",
         year == 2007) |>
  mutate(pop_thousands = pop/1000) |>
  dplyr::select(country, year, pop_thousands) #this cleans up the dataframe but isn't necessary

> Then, sync to Github.com (pull, stage, commit, push).

---
### Functions in `mutate()`

- arithmetic operators: `+, -, *, /, ^`
- modular arithmetic: `%/%, %%`
- logs: `log(), log2(), log10()`
- offsets: `lead(), lag()`
- cumulative and rolling aggregates: `cumsum(), cumprod(), cummin(), cummax(), cummean()`
- logical comparisons: `<, <=, >, >=, !=`
- ranking: `min_rank(), row_number(), dense_rank(), percent_rank(), cume_dist(), ntile()`

- Any R or custom function that returns a **vector** with the same length as the number of rows
---
### `group_by()` operates on groups

What if we wanted to know the total population on each continent in 2002? Answering this question requires a **grouping variable**.

Visually, we are doing this (thanks RStudio for your cheatsheet):

In [ ]:
knitr::include_graphics('figures/rstudio-cheatsheet-group_by.png')

---
By using `group_by()` we can set our grouping variable to `continent` and create a new column called `cont_pop` that will add up all country populations by their associated continents.

In [ ]:
gapminder |>
  filter(year == 2002) |>
  group_by(continent) |> 
  mutate(cont_pop = sum(pop))

---
What if we don’t care about the other columns and we only want each continent and their population in 2002? Here’s the next function:

### `summarize()` with `group_by()`

We want to operate on a group, but actually collapse or distill the output from that group. The `summarize()` function will do that for us.

Visually, we are doing this (thanks RStudio for your cheatsheet):

In [ ]:
knitr::include_graphics('figures/rstudio-cheatsheet-summarise.png')

---
Here we go:

In [ ]:
gapminder |>
  group_by(continent) |>
  summarize(cont_pop = sum(pop)) |>
  ungroup()

`summarize()` will actually only keep the columns that are `grouped_by` or summarized. So if we wanted to keep other columns, we’d have to do have a few more steps. `ungroup()` removes the grouping and it’s good to get in the habit of using it after a `group_by()`.

---
We can use more than one grouping variable. Let’s get total populations by continent and year.

In [ ]:
gapminder |>
  group_by(continent, year) |>
  summarize(cont_pop = sum(pop))

---
### Functions in `summarise()`

- location: `mean(x), median(x)`
- spread: `sd(x), IQR(x), mad(x)`
- rank: `min(x), quantile(x, 0.25), max(x)`
- position: `first(x), nth(x, 2), last(x)`
- count: `n(x), sum(!is.na(x)), n_distinct(x)`
- any base R or custom function that returns **one summary value**

---
### arrange() orders columns
This is ordered alphabetically, which is cool. But let’s say we wanted to order it in ascending order for year. The dplyr function is arrange().

In [ ]:
gapminder |>
  group_by(continent, year) |>
  summarize(cont_pop = sum(pop)) |>
  arrange(year)

- Use `desc(var)` to arrange in decreasing order
- Possible to use multiple variables (categorical)
---
#### Your turn
> What is the maximum GDP per continent across all years?

In [ ]:
gapminder |>
  mutate(gdp = pop * gdpPercap) |>
  group_by(continent) |>
  mutate(max_gdp = max(gdp)) |>
  filter(gdp == max_gdp)

#### Your turn
> 1. arrange your data frame in descending order (opposite of what we’ve done). Expect that this is possible: ?arrange
> 2. save your data frame as a variable
> 3. find the maximum life expectancy for countries in Asia. What is the earliest year you encounter? The latest? Hint: you can use or `base::max` and `dplyr::arrange()`
> 4. Knit your RMarkdown file, and sync it to GitHub (pull, stage, commit, push)

In [ ]:
asia_life_exp <- gapminder |>
  filter(continent == 'Asia') |>
  group_by(country) |>
  filter(lifeExp == max(lifeExp)) |>
  arrange(year) 

---
### All together now
We have done a pretty incredible amount of work in a few lines. Our whole analysis is this. Imagine the possibilities from here. It’s very readable: you see the data as the first thing, it’s not nested. Then, you can read the verbs. This is the whole thing, with explicit package calls from `readr::` and `dplyr::`:

In [ ]:
## load libraries
library(tidyverse) ## install.packages('tidyverse')

## read in data
gapminder <- readr::read_csv('data/gapminder.csv') 

## summarize
gap_max_life_exp <- gapminder |> 
  dplyr::select(-continent, -lifeExp) |> # or select(country, year, pop, gdpPercap)
  dplyr::group_by(country) |>
  dplyr::mutate(gdp = pop * gdpPercap) |>
  dplyr::summarize(max_gdp = max(gdp)) |>
  dplyr::ungroup() 

---
### Compare to base R

Instead of calculating the max for each country like we did with dplyr above, here we will calculate the max for one country, Mexico.

In [ ]:
gapminder <- read.csv('data/gapminder.csv', stringsAsFactors = FALSE) 
x1  <- gapminder[ , c('country', 'year', 'pop', 'gdpPercap') ]# subset columns
mex <- x1[x1$country == "Mexico", ] # subset rows
mex$gdp <- mex$pop * mex$gdpPercap # add new columns
mex$max_gdp <- max(mex$gdp)

Note too that the chain operator `|>` that we used with the `tidyverse` lets us get away from the temporary variable `x1`.

---
## Joining datasets
Most of the time you will have data coming from different places or in different files, and you want to put them together so you can analyze them. Datasets you’ll be joining can be called relational data, because it has some kind of relationship between them that you’ll be acting upon. In the `tidyverse`, combining data that has a relationship is called “joining”.

From the RStudio cheatsheet (note: this is an earlier version of the cheatsheet but I like the graphics):

In [ ]:
knitr::include_graphics('figures/rstudio-cheatsheet-combine.png')

---
Let’s have a look at this and pretend that the x1 column is a study site and x2 is the variables we’ve recorded (like species count) and x3 is data from an instrument (like temperature data). Notice how you may not have exactly the same observations in the two datasets: in the x1 column, observations A and B appear in both datasets, but notice how the table on the left has observation C, and the table on the right has observation D.
---
If you wanted to combine these two tables, how would you do it? There are some decisions you’d have to make about what was important to you. The cheatsheet visualizes it for us:

In [ ]:
knitr::include_graphics('figures/rstudio-cheatsheet-combine-options1.png')

---
We will only talk about this briefly here, but you can refer to this more as you have your own datasets that you want to join. This describes the figure above:

- `left_join` keeps everything from the left table and matches as much as it can from the right table. In R, the first thing that you type will be the left table (because it’s on the left)
- `right_join` keeps everything from the right table and matches as much as it can from the left table
- `inner_join` only keeps the observations that are similar between the two tables
- `full_join` keeps all observations from both tables.

---
Let’s play with these CO2 emissions data to illustrate:

In [ ]:
## read in the data. (same URL as yesterday, with co2.csv instead of gapminder.csv)
co2 <- read_csv("data/co2.csv")

## explore
co2 |> head()
co2 |> dim() # 12

---

In [ ]:
## create new variable that is only 2007 data
gap_2007 <- gapminder |>
  filter(year == 2007) 
gap_2007 |> dim() # 142  

## left_join gap_2007 to co2
lj <- left_join(gap_2007, co2, by = "country")

## explore
lj |> dim() #142
lj |> head(3) # lots of NAs in the co2_2017 columm

---

In [ ]:
## right_join gap_2007 and co2
rj <- right_join(gap_2007, co2, by = "country")

## explore
rj |> dim() # 12
rj |> head(3)

---
### Key Points
Data manipulation functions in `dplyr` allow you to `filter()` by rows and `select()` by columns, create new columns with `mutate()`, and `group_by()` unique column values to apply `summarize()` for new columns that define aggregate values across groupings.
The “then” operator `|>` allows you to chain successive operations without needing to define intermediary variables for creating the most parsimonious, easily read analysis.

#### Error: unexpected SPECIAL in "`|>`"
If you get this error, it is probably because you have a line that starts with a pipe. The pipe should be at the end of the previous line, not the start of the current line.

**Yes:**

In [ ]:
gap_cambodia  <- gapminder |> filter(country == "Cambodia") |>
  select(-continent, -lifeExp)

**No:**

In [ ]:
gap_cambodia  <- gapminder |> filter(country == "Cambodia") 
  |> select(-continent, -lifeExp)
# Error: unexpected SPECIAL in "  |>"

In [ ]:
gap_china  <- gapminder |> filter(country == "China") |>
  select(-continent, -lifeExp) |> mutate(gdp = pop * gdpPercap, loggdp = log(gdp))
gap_china |> ggplot(aes(x = year, y = gdp))+ geom_point()+
  geom_line(col = 'red')
gap_china |> ggplot(aes(x = year, y = loggdp))+ geom_point()+
  geom_line(col = 'red')

---
## Data Wrangling: `tidyr`

Now you have some experience working with tidy data and seeing the logic of wrangling when data are structured in a tidy way. But ‘real’ data often don’t start off in a tidy way, and require some reshaping to become tidy. The `tidyr` package is for reshaping data. You won’t use `tidyr` functions as much as you use dplyr functions, but it is incredibly powerful when you need it.

Why is this important? Well, if your data are formatted in a standard way, you will be able to use analysis tools that operate on that standard way. Your analyses will be streamlined and you won’t have to reinvent the wheel every time you see data in a different.

Data are often entered in a _wide_ format where each row is often a site/subject/patient and you have multiple observation variables containing the same type of data.
---
An example of data in a _wide_ format is the `AirPassengers` dataset which provides information on monthly airline passenger numbers from 1949-1960. You’ll notice that each row is a single year and the columns are each month Jan - Dec.

In [ ]:
AirPassengers

---
This format is intuitive for data entry, but less so for data analysis. If you wanted to calculate the monthly mean, where would you put it? As another row?

Often, data must be reshaped for it to become tidy data. What does that mean? There are four main verbs we’ll use, which are essentially pairs of opposites:

- turn columns into rows (`gather()`),
- turn rows into columns (`spread()`),
- turn a character column into multiple columns (`separate()`),
- turn multiple character columns into a single column (`unite()`)

In [ ]:
knitr::include_graphics('figures/rstudio-cheatsheet-spread-gather-sep-unite.png')

---
### Explore gapminder dataset
First have a look at the _wide_ format data.

You can see there are a lot more columns than the version we looked at before. This format is pretty common, because it can be a lot more intuitive to enter data in this way.

In [ ]:
knitr::include_graphics('figures/gapminder_wide_gh.png')

---
### Setup
We’ll learn `tidyr` in an RMarkdown file within a GitHub repository so we can practice what we’ve learned so far. You can either continue from the same RMarkdown as yesterday, or begin a new one.

**Here’s what to do**:

1. Clear your workspace (Session > Restart R)
2. New File > R Markdown..., save as something other than `gapminder-wrangle.Rmd` and delete irrelevant info, or just continue using `gapminder-wrangle.Rmd`
---
#### load `tidyverse` (which has `tidyr` inside)
First load `tidyr` in an R chunk. You already have installed the tidyverse, so you should be able to just load it like this (using the comment so you can run `install.packages("tidyverse")` easily if need be):

In [ ]:
library(tidyverse) # install.packages("tidyverse")

### `gather()` data from wide to long format

In [ ]:
knitr::include_graphics('figures/rstudio-cheatsheet-reshaping-data-gather.png')

---
Read in the data. Let’s also read in the gapminder data from yesterday so that we can use it to compare later on.

In [ ]:
## wide format
gap_wide <- readr::read_csv('data/gapminder_wide.csv')
gapminder <- readr::read_csv('data/gapminder.csv')

Let’s have a look:

In [ ]:
#head(gap_wide)
str(gap_wide)

---
While wide format is nice for data entry, it’s not nice for calculations. Some of the columns are a mix of variable (e.g. “gdpPercap”) and data (“1952”). What if you were asked for the mean population after 1990 in Algeria? Possible, but ugly. But we know it doesn’t need to be so ugly. Let’s tidy it back to the format we’ve been using.

> Question: let’s talk this through together. If we’re trying to turn the `gap_wide` format into `gapminder` format, what structure does it have that we like? And what do we want to change?

- We like the continent and country columns. We won’t want to change those.
- We want 1 column identifying the variable name (`tidyr` calls this a **‘key’**), and 1 column for the data (`tidyr` calls this the **’value’**).
- We actually want 3 different columns for variable: `gdpPercap`, `lifeExp`, and `pop`.
- We would like year as a separate column.
---
Let’s get it to long format. We’ll have to do this in 2 steps. The first step is to take all of those column names (e.g. `lifeExp_1970`) and make them a variable in a new column, and transfer the values into another column. 

> Question: What is our **key-value pair**?

We need to name two new variables in the key-value pair, one for the key, one for the value. It can be hard to wrap your mind around this, so let’s give it a try. Let’s name them `obstype_year` and `obs_values`.

Here’s the start of what we’ll do:

In [ ]:
gap_long <- gap_wide |> 
  gather(key   = obstype_year,
         value = obs_values)

Let's inspect our work.

In [ ]:
str(gap_long)

---

In [ ]:
head(gap_long)
tail(gap_long)

---
We have reshaped our dataframe but this new format isn’t really what we wanted.

What went wrong? Notice that it didn’t know that we wanted to keep `continent` and `country` untouched; we need to give it more information about which columns we want reshaped. We can do this in several ways.

One way is to identify the columns is by name. Listing them explicitly can be a good approach if there are just a few. But in our case we have 30 columns. I’m not going to list them out here since there is way too much potential for error if I tried to list `gdpPercap_1952`, `gdpPercap_1957, gdpPercap_1962` and so on. But we could use some of `dplyr`’s awesome helper functions — because we expect that there is a better way to do this!

In [ ]:
gap_long <- gap_wide |> 
  gather(key   = obstype_year,
         value = obs_values,
         dplyr::starts_with('pop'),
         dplyr::starts_with('lifeExp'),
         dplyr::starts_with('gdpPercap'))  #here i'm listing all the columns to use in gather
str(gap_long)

---

In [ ]:
head(gap_long)
tail(gap_long)

---
Success! And there is another way that is nice to use if your columns don’t follow such a structured pattern: you can exclude the columns you don’t want.

In [ ]:
gap_long <- gap_wide |> 
  gather(key   = obstype_year,
         value = obs_values,
         -continent, -country)

str(gap_long)
head(gap_long, 3)

---
To recap:

Inside `gather()` we first name the new column for the new ID variable (`obstype_year`), the name for the new amalgamated observation variable (`obs_value`), then the names of the old observation variable. We could have typed out all the observation variables, but as in the `select()` function (see `dplyr` lesson), we can use the `starts_with()` argument to select all variables that starts with the desired character string. Gather also allows the alternative syntax of using the - symbol to identify which variables are not to be gathered (i.e. ID variables).

OK, but we’re not done yet. `obstype_year` actually contains two pieces of information, the observation type (`pop`,`lifeExp`, or `gdpPercap`) and the year. We can use the `separate()` function to split the character strings into multiple variables.
---
`?separate` –> the main arguments are `separate(data, col, into, sep ...)`. So we need to specify which column we want separated, name the new columns that we want to create, and specify what we want it to separate by. Since the `obstype_year` variable has observation types and years separated by a `_`, we’ll use that.

In [ ]:
gap_long <- gap_wide |> 
  gather(key   = obstype_year,
         value = obs_values,
         -continent, -country) |>
  separate(obstype_year,
           into = c('obs_type','year'),
           sep = "_",
           convert = TRUE) #this ensures that the year column is an integer rather than a character
str(gap_long)

Excellent. This is long format: every row is a unique observation. 

---
### Plot long format data
The long format is the preferred format for plotting with `ggplot2`. Let’s look at an example by plotting just Canada’s life expectancy.

In [ ]:
canada_df <- gap_long |>
  filter(obs_type == "lifeExp",
         country == "Canada")
ggplot(canada_df, aes(x = year, y = obs_values)) +
  geom_line()

In [ ]:
cambodia_df <- gap_long |>
  filter(obs_type == "lifeExp",
         country == "Cambodia")
ggplot(cambodia_df, aes(x = year, y = obs_values)) +
  geom_line()

gap_long |>
  filter(obs_type == "lifeExp",
         country == "China") |> 
ggplot(aes(x = year, y = obs_values)) +
  geom_point()+
  geom_line(col = 'blue')

---
We can also look at all countries in the Americas:

In [ ]:
life_df <- gap_long |>
  filter(obs_type == "lifeExp",
         continent == "Americas")
ggplot(life_df, aes(x = year, y = obs_values, color = country)) +
  geom_line()

---
### Exercise
Using `gap_long`, calculate and plot the the mean life expectancy for each continent over time from 1982 to 2007. Give your plot a title and assign x and y labels. Hint: do this in two steps. First, do the logic and calculations using `dplyr::group_by()` and `dplyr::summarize()`. Second, plot using `ggplot()`.

In [ ]:
continents <- gap_long |> 
  filter(obs_type == "lifeExp", 
         year > 1980) |> 
  group_by(continent, year) |> 
  summarize(mean_le = mean(obs_values)) |>
  ungroup()

ggplot(data = continents, aes(x = year, y = mean_le, color = continent)) + 
  geom_line() +
  labs(title = "Mean life expectancy",
       x = "Year",
       y = "Age (years)") 

## Additional customization
ggplot(data = continents, aes(x = year, y = mean_le, color = continent)) + 
  geom_line() +
  labs(title = "Mean life expectancy",
       x = "Year",
       y = "Age (years)",
       color = "Continent") +
  theme_classic() +
  scale_fill_brewer(palette = "Blues")   

---
### `spread()`
The function `spread()` is used to transform data from long to wide format

Alright! Now just to double-check our work, let’s use the opposite of `gather()` to spread our observation variables back to the original format with the aptly named `spread()`. You pass `spread()` the key and value pair, which is now `obs_type` and `obs_values`.

In [ ]:
knitr::include_graphics('figures/rstudio-cheatsheet-reshaping-data-spread.png')

In [ ]:
gap_normal <- gap_long |> 
  spread(obs_type, obs_values)

In [ ]:
canada_df <- gap_normal |>
  filter(country == "Canada") 
ggplot(canada_df, aes(x = gdpPercap, y = lifeExp)) +
  geom_point()

---

In [ ]:
dim(gap_normal)
dim(gapminder)
names(gap_normal)
names(gapminder)

Now we’ve got a dataframe `gap_normal` with the same dimensions as the original `gapminder`.
---
### Exercise
1. Convert `gap_long` all the way back to `gap_wide`. Hint: Do this in 2 steps. First, create appropriate labels for all our new variables (variable_year combinations) with the opposite of separate: `tidyr::unite()`. Second, `spread()` that variable_year column into wider format.

In [ ]:
head(gap_long) # remember the columns

gap_wide_new <- gap_long |> 
  # first unite obs_type and year into a new column called var_names. Separate by _
  unite(col = var_names, obs_type, year, sep = "_") |> 
  # then spread var_names out by key-value pair.
  spread(key = var_names, value = obs_values)
str(gap_wide_new)

2. Knit the R Markdown file and sync to Github (pull, stage, commit, push)

### clean up and save your .Rmd
Spend some time cleaning up and saving `gapminder-wrangle.Rmd` Restart R. In RStudio, use _Session > Restart R_. Otherwise, quit R with `q()` and re-launch it.

---
### Is there a relationship between life expectancy and GDP per capita?

Let's use the gapminder data to answer this question. To try and answer it we will make a scatterplot. We will do this for the latest entry in the dataset which is:

In [ ]:
max( gapminder$year )

In [ ]:
gapminder |> filter(year==2007) |> 
  ggplot(aes(x = gdpPercap, y = lifeExp))+ 
  geom_point()+
  labs(x = "GDP per Capita", 
       y = "Life Expectancy", 
       title = "Data from Gapminder") 

---

In [ ]:
knitr::include_graphics('figures/gap07.png')

---
From this plot, we see that there is a wide variability in life expectancy for the lower income countries and then somewhat of a positive trend. However, there are many countries with incomes below 5,000 dollars per person and it is hard to see differences between these.

We can examine just this variable with a histogram. 

In [ ]:
gapminder |> 
    filter(year==2007) |> 
    ggplot(aes(x=gdpPercap)) + geom_histogram()

---
The histogram shows very large tails. We may do better by transforming the data. For data like this, the log transformation seems to work well. It also has a nice economic interpretation related to percent growth: in $\log_{10}$ a change of 1 means the country is 10 times richer.

So how do we make the x-axis in the log scale? It is convenient to have this cheat sheet around when using `ggplot2`. From there we see that `scale_x_log10` does what we want.

In [ ]:
gapminder |> 
    filter(year==2007) |> 
    ggplot(aes(x=gdpPercap)) + 
    geom_histogram(bins=25) + scale_x_log10()

---
We no longer see extreme tails. The scatter plot now looks much more informative:

Now we can remake the scatter plot but now make sure the x-axis is in a log-scale.

In [ ]:
gapminder |> 
    filter(year==2007) |> 
    ggplot(aes(x=gdpPercap, y = lifeExp)) +
    geom_point() + scale_x_log10()

---
We can also use other really great packages, such as `ggrepel`:

In [ ]:
library(ggrepel)
gapminder |> 
    filter(year==2007) |> 
    ggplot(aes(x=gdpPercap, y = lifeExp)) +
    geom_point(color = 'red',) + scale_x_log10() + 
    geom_text_repel(aes(label = country), size = 2) +
  theme_classic() 

---

In [ ]:
knitr::include_graphics('figures/gap07_repel.png')